In [28]:
import json

# Formatting data for training/testing
with open('contracts.json', 'r') as file:
    data = json.load(file)

formatted_data = []

for i, entry in enumerate(data):
    context = entry['Context']
    qas = []
    for q, a in entry.items():
        if q != "Context" and q != "Date":
            answer_start = context.find(a)
            if answer_start != -1:
                qas.append({
                    "id": str(i+1) + str(len(qas) + 1).zfill(5),
                    "is_impossible": False,
                    "question": q,
                    "answers": [
                        {
                            "text": a,
                            "answer_start": answer_start,
                        }
                    ],
                })
    formatted_data.append({
        "context": context,
        "qas": qas,
    })

In [29]:
from sklearn.model_selection import train_test_split

# Split data and save into separate JSONs
train_data, test_data = train_test_split(formatted_data, test_size=0.10, random_state = 90)

with open('contract_data_train.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

with open('contract_data_test.json', 'w', encoding='utf-8') as f:
    json.dump(test_data, f, ensure_ascii=False, indent=4)

In [30]:
!pip install simpletransformers




In [31]:
import logging
import re
import copy
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

class ContractQAModel:
    def __init__(self, model_type, model_name, train_args, use_cuda):
        self.model = QuestionAnsweringModel(model_type,
                                            model_name,
                                            args = train_args,
                                            use_cuda = use_cuda)

    @staticmethod
    def post_process_answer(answer):
        # Remove leading and trailing punctuation, whitespace and
        # special characters, except for $ at beginning
        cleaned_answer = re.sub(r'^[(.,\s]+|[).,\s]+$', '', answer)
        return cleaned_answer

    @staticmethod
    def validate_answer(truth, prediction, threshold):

        def calculate_overlap(string1, string2):
            overlap_length = min(len(string1), len(string2))
            for i in range(overlap_length, 0, -1):
                if string1[-i:] == string2[:i]:
                    return i
            return 0

        overlap1 = calculate_overlap(truth, prediction)
        overlap2 = calculate_overlap(prediction, truth)

        if overlap1 >= threshold * min(len(truth), len(prediction)) or overlap2 >= threshold * min(len(prediction), len(truth)):
            return True
        else:
            return False


    def train_model(self, train_data, **kwargs):
        self.model.train_model(train_data, **kwargs)

    def predict(self, text_data, n_best_size):
        predictions, raw_outputs = self.model.predict(text_data, n_best_size)

        # post-processing
        for prediction in predictions:
            pp_answers = []
            for a in prediction['answer']:
                pp_answers.append(self.post_process_answer(a))
            prediction['answer'] = pp_answers

        return predictions, raw_outputs

    def evaluate_model(self, eval_data, **kwargs):
        # result, outputs = self.model.eval_model(eval_data, **kwargs)
        threshold = 0.7

        eval_data_copy = copy.deepcopy(eval_data)

        predictions, raw_outputs = self.predict(eval_data_copy, n_best_size=1)

        qas = []
        for item in eval_data:
            qas.extend(item['qas'])

        num_correct = 0
        correct = []
        incorrect = []

        for prediction, qa in zip(predictions, qas):
            truth = qa['answers'][0]['text']
            pred = prediction['answer'][0]

            if self.validate_answer(truth, pred, threshold):
                num_correct += 1
                correct.append({
                  "question": qa['question'],
                  "id": qa['id'],
                  "truth": truth,
                  "prediction": pred,
                })
            else:
                incorrect.append({
                  "question": qa['question'],
                  "id": qa['id'],
                  "truth": truth,
                  "prediction": pred,
                })

        accuracy = num_correct / len(qas)

        print("num correct:     " + str(num_correct))
        print("num incorrect:   " + str(len(qas) - num_correct))
        print("accuracy:        " + str(accuracy))

        outputs = {"correct": correct, "incorrect": incorrect}

        return outputs

In [32]:
# Open JSONs
with open(r"contract_data_train.json", "r") as read_file:
    train = json.load(read_file)

with open(r"contract_data_test.json", "r") as read_file:
    test = json.load(read_file)

In [33]:
# Training parameters
train_args = {
    'overwrite_output_dir': True,
    "evaluate_during_training": False,
    "max_seq_length": 128,
    "num_train_epochs": 3,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":16,
    "train_batch_size": 4,
    "eval_batch_size": 4,
    "learning_rate": 0.00005,
    "weight_decay": 0.01,
    "do_lower_case": True

}

# Model initialization
model1 = ContractQAModel("roberta",
                        "deepset/roberta-base-squad2",
                        train_args = train_args,
                        use_cuda=True)

In [34]:
# Train model
model1.train_model(train)

add example index and unique id: 100%|██████████| 189/189 [00:00<00:00, 371647.19it/s]


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/48 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 2 of 3:   0%|          | 0/48 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/48 [00:00<?, ?it/s]

In [35]:
# Evaluate the model
outputs1 = model1.evaluate_model(test)

add example index and unique id: 100%|██████████| 21/21 [00:00<00:00, 126734.37it/s]


Running Prediction:   0%|          | 0/6 [00:00<?, ?it/s]

num correct:     18
num incorrect:   3
accuracy:        0.8571428571428571


In [36]:
print(outputs1)

{'correct': [{'question': 'What is the contract number?', 'id': '1800001', 'truth': 'W9127N-20-C-0028', 'prediction': 'W9127N-20-C-0028'}, {'question': 'What is the company that was awarded the contract?', 'id': '1800002', 'truth': 'Oregon Department of Fish and Wildlife', 'prediction': 'Oregon Department of Fish and Wildlife'}, {'question': 'What is the amount awarded?', 'id': '1800003', 'truth': '$7,887,876', 'prediction': '$7,887,876'}, {'question': 'What is the contract awarded for?', 'id': '1800004', 'truth': 'electric power support', 'prediction': 'electric power support'}, {'question': 'Where will the work be performed?', 'id': '1800005', 'truth': 'Trail, Oregon', 'prediction': 'Trail, Oregon'}, {'question': 'When is the contract expected to be completed by?', 'id': '1800006', 'truth': 'Aug. 31, 2023', 'prediction': 'Aug. 31, 2023'}, {'question': 'Who is the contracting activity organization?', 'id': '1800007', 'truth': 'U.S. Army Corps of Engineers', 'prediction': 'U.S. Army Co

In [37]:
# Make predictions with the model
to_predict = [
    {
        "context": "Bren-Tronics Inc.,* Commack, New York, has been awarded an estimated $9,708,829 fixed-price with economic-price-adjustment, indefinite-quantity contract for lithium-ion batteries used in multiple communications platforms. This was a competitive acquisition with four responses received. This is a three-year base contract with two one-year option periods. Location of performance is New York, with an April 22, 2027, performance completion date. Using military services are Army, Navy and Marine Corps. Type of appropriation is fiscal 2024 through 2027 defense working capital funds. The contracting activity is the Defense Logistics Agency Land and Maritime, Columbus, Ohio (SPE7LX-24-D-0042).",
        "qas": [
            {
                "question": "Where will the work be performed?",
                "id": "0",
            },
        ],
    }
]

predictions, raw_outputs = model1.predict(to_predict, n_best_size=2)
print(predictions)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 10180.35it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['New York', 'New York, has']}]
